In [2]:
import torch
from torch import nn
from torch.nn import functional as F

In [3]:
# 多层感知机
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

In [4]:
X = torch.rand(2, 20)
net(X)

tensor([[-0.2329, -0.2619,  0.0418, -0.0495, -0.2186,  0.2657,  0.0043, -0.0062,
         -0.0433,  0.1511],
        [-0.3801, -0.2870, -0.1477, -0.0614, -0.3492,  0.5978,  0.2029,  0.1652,
          0.0050,  0.0579]], grad_fn=<AddmmBackward0>)

# 自定义块

In [5]:
class MLP(nn.Module):
    # ⽤模型参数声明层。这⾥，我们声明两个全连接的层
    def __init__(self):
        # 调⽤MLP的⽗类Module的构造函数来执⾏必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)    # 隐藏层
        self.out = nn.Linear(256, 10)       # 输出层
    
    # 定义模型的前向传播，即如何根据输⼊X返回所需的模型输出
    def forward(self, X):
        # 注意，这⾥我们使⽤ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [6]:
net = MLP()
net(X)

tensor([[ 0.1631,  0.0315,  0.1008,  0.0366, -0.2807, -0.1066, -0.1764, -0.1203,
         -0.0209,  0.1461],
        [ 0.2451, -0.0300,  0.0800,  0.1716, -0.2833,  0.0251, -0.2061, -0.0387,
          0.0070,  0.0258]], grad_fn=<AddmmBackward0>)

# 顺序块

In [7]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这⾥，module是Module⼦类的⼀个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module
    
    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [8]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1946, -0.0299, -0.0490, -0.1135,  0.0295,  0.0570,  0.1820,  0.0717,
         -0.1548,  0.0352],
        [ 0.2568, -0.0255, -0.1142, -0.2377, -0.0929,  0.0987, -0.0822,  0.0214,
         -0.1581, -0.0782]], grad_fn=<AddmmBackward0>)

# 在前向传播函数中执⾏代码

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        # 使⽤创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复⽤全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [10]:
net = FixedHiddenMLP()
net(X)

tensor(0.3229, grad_fn=<SumBackward0>)

混合搭配各种组合块的⽅法

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

In [12]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0826, grad_fn=<SumBackward0>)